In [ ]:
using Revise, Genie, KitMSA, PyPlot, JLD2, Statistics, StatsBase, DCAUtils, DelimitedFiles, CSV, DataFrames

MSA_train = Int8.(KitMSA.fasta2matrix("../files_nobu_evolution_exp/natural_seqs/B1_all_Sevan_hmm03_max010rowgaps_max075flankcolgaps_max080wtid.fasta")')
MSA_train2 = Int8.(KitMSA.fasta2matrix("../files_nobu_evolution_exp/natural_seqs/B1_all_Sevan_hmm03_max010rowgaps_max075flankcolgaps.fasta")')
@load "../data_Genie/pars_beta_B1_integrated.jld2"

w_fra = readdlm("../data_Genie/Weights_leonardo.dat")

L,M = size(MSA_train); q = 21;
VIM2 = Int.(KitMSA.fasta2matrix("../files_nobu_evolution_exp/wts/VIM2_V185I_hmmstruct_symfrac03_max075flankcolgaps.fasta"));

# NDM-1 (there is a DMS of this sequence)
NDM1 = Int.(KitMSA.fasta2matrix("../files_nobu_evolution_exp/wts/NDM1_hmmstruct_symfrac03_max075flankcolgaps.fasta"));


dms_map = CSV.read("metallo_beta/Dataframe_matching_mutations_model_removed100_9_2024.txt", DataFrame; delim=',');
pos_map = CSV.read("metallo_beta/simple_matching_positions_DCAmodel_experiments_VIM_NDM.csv", DataFrame);
revers = CSV.read("metallo_beta/DCA effect reversion latest model.txt", DataFrame; delim='\t')
double = CSV.read("metallo_beta/DCA effect double mutants.txt", DataFrame; delim='\t')

ndm_r2mod = Dict(pos_map[:, 1] .=> pos_map[:, 2])
ndm_mod2r = Dict(pos_map[:, 2] .=> pos_map[:, 1])

vim_r2mod = Dict(pos_map[:, 4] .=> pos_map[:, 5])
vim_mod2r = Dict(pos_map[:, 5] .=> pos_map[:, 4]);

mod_ndm2vim = Dict(pos_map[:, 2] .=> pos_map[:, 5])
mod_vim2ndm = Dict(pos_map[:, 5] .=> pos_map[:, 2]);

## Greedy

In [ ]:
greedy_msa = Int.(read_fasta_alignment("metallo_beta/path_greedy_VIM2_NDM1_27",0.9));

mean_msa = Int.(read_fasta_alignment("metallo_beta/path_meanE_1_27",0.9));

mean_full_msa = Int.(read_fasta_alignment("metallo_beta/full_paths/path_meanE_beta_5_n_1",0.9));
 
ps_ij= []; ens = []; dists =[]; 

ps_ij2= []; ens2 = []; dists2 =[]; 

ps_ij_full= []; ens_full = []; dists_full =[];

T = 1.; n_subs = 130; for i in 1:8
    mut1 = double.Mutation_1[i]
    mut2 = double.Mutation_2[i]
   
    a1i = KitMSA.letter2num(first(mut1))
    a2i = KitMSA.letter2num(first(mut2))
    
    a1f = KitMSA.letter2num(last(mut1))
    a2f = KitMSA.letter2num(last(mut2))
    m = match(r"\d+", mut1)
    site1 = ndm_r2mod[parse(Int, m.match)]
    m = match(r"\d+", mut2)
    site2 = ndm_r2mod[parse(Int, m.match)]
    
    println("$(double.Mutation_1[i]) + $(double.Mutation_2[i])")
    
    ndm_p = Genie.dist_proba_2_point(site1, site2, NDM1, h, J, L, q, 1.)
    vim_p = Genie.dist_proba_2_point(site1, site2, VIM2, h, J, L, q, 1.)
    
    println("NDM1 i->f")
    println([ndm_p[a1i, a2i], ndm_p[a1f, a2f]])
    println("VIM2 i->f")
    println([vim_p[a1i, a2i], vim_p[a1f, a2f]])
      
    push!(ens, Genie.energy(greedy_msa, h, J))
    push!(ps_ij, [Genie.dist_proba_2_point(site1, site2, greedy_msa[:,t],
            h, J, L, q, 1.)[a1f,a2f] for t in 1:size(greedy_msa,2)])
    push!(dists, ham_dist(NDM1, greedy_msa))
    
    push!(ens_full, Genie.energy(mean_full_msa, h, J))
    push!(ps_ij_full, [Genie.dist_proba_2_point(site1, site2, mean_full_msa[:,t],
            h, J, L, q, 1.)[a1f,a2f] for t in 1:size(mean_full_msa,2)])
    push!(dists_full, ham_dist(NDM1, mean_full_msa))
    
    push!(ens2, Genie.energy(mean_msa, h, J))
    push!(ps_ij2, [Genie.dist_proba_2_point(site1, site2, mean_msa[:,t],
            h, J, L, q, 1.)[a1f,a2f] for t in 1:size(mean_msa,2)])
    push!(dists2, ham_dist(NDM1, mean_msa))
end

color_names = ["red", "orange", "yellow", "green", "blue", "purple", "brown","gray"]; 

close("all")
for i in 1:length(dists)
    #plt.plot(dists[i], ps_ij[i], color = color_names[i], label = " Greedy $(double.Mutation_1[i]) + $(double.Mutation_2[i])")
    plt.plot(dists2[i], ps_ij2[i], color = color_names[i], linestyle  = "--", label = " Mean $(double.Mutation_1[i]) + $(double.Mutation_2[i])")
    plt.plot(dists_full[i], ps_ij_full[i], color = color_names[i],  label = " Mean Full $(double.Mutation_1[i]) + $(double.Mutation_2[i])")
end 
plt.xlabel("# reversions")
plt.ylabel("P_ij")
plt.legend()
savefig("../mean_p_ij_muts.png")


close("all")
plt.scatter(dists[1], ens[1], label = "Greedy")
plt.scatter(dists2[1], ens2[1], label = "Mean")
plt.scatter(dists_full[1], ens_full[1], label = "Mean full")
plt.xlabel("# reversions")
plt.ylabel("DCA energy")
plt.legend()
savefig("../mean_ens_muts.png")





In [ ]:
ps_ij= []; ens = []; dists =[]; 

ps_ij2= []; ens2 = []; dists2 =[]; 

ps_ij_full= []; ens_full = []; dists_full =[];

T = 1.; n_subs = 130; for i in 1:8
    mut1 = double.Mutation_1[i]
    mut2 = double.Mutation_2[i]
   
    a1i = KitMSA.letter2num(first(mut1))
    a2i = KitMSA.letter2num(first(mut2))
    
    a1f = KitMSA.letter2num(last(mut1))
    a2f = KitMSA.letter2num(last(mut2))
    m = match(r"\d+", mut1)
    site1 = ndm_r2mod[parse(Int, m.match)]
    m = match(r"\d+", mut2)
    site2 = ndm_r2mod[parse(Int, m.match)]
    
    println("$(double.Mutation_1[i]) + $(double.Mutation_2[i])")
    
    ndm_p = Genie.dist_proba_2_point(site1, site2, NDM1, h, J, L, q, 1.)
    vim_p = Genie.dist_proba_2_point(site1, site2, VIM2, h, J, L, q, 1.)
    
    println("NDM1 i->f")
    println([ndm_p[a1i, a2i], ndm_p[a1f, a2f]])
    println("VIM2 i->f")
    println([vim_p[a1i, a2i], vim_p[a1f, a2f]])
      
    push!(ens2, Genie.energy(mean_msa, h, J))
    push!(ps_ij2, [Genie.dist_proba_2_point(site1, site2, mean_msa[:,t],
            h, J, L, q, 1.)[a1i,a2i] for t in 1:size(mean_msa,2)])
    push!(dists2, ham_dist(NDM1, mean_msa))
end
